In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# Load WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents from JSON file
data_file = open('drone.json', encoding='utf-8').read()
intents = json.loads(data_file)

# Initialize lists for words, classes, and documents
words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Preprocess intents data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize words and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
                                                                         
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Save words and classes to pickle files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Initializing training data
training = []
output_empty = [0] * len(classes)

# Iterate through documents to create training data
for doc in documents:
    bag = []

    # Lemmatize pattern words and create bag-of-words representation
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Create output row (one-hot encoding for the class)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Append to training data
    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)

# Convert training data to NumPy array
training = np.array(training, dtype=object)  # Set dtype to object

# Extract patterns and intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Create and compile the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit and save the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=20000, batch_size=8, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created and trained successfully.")

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('drone.json', encoding='utf-8').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
        else:
            result = "You must ask the right questions"
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [4]:
def drone_working_hours():
    return "Drones typically have a working time ranging from 20 to 30 minutes per battery charge. The exact duration may vary based on the drone model and usage."

def drone_price():
    return "The price of drones can vary significantly depending on the brand, model, and features. Entry-level drones may start around $100, while professional-grade drones with advanced capabilities can cost several thousand dollars."

def drone_taxi_services():
    return "Drone taxis are a futuristic mode of transportation where passengers are transported by unmanned aerial vehicles. Imagine skipping traffic and reaching your destination in the sky! While still in the experimental phase, drone taxi services promise a new era of convenient and fast travel."

def drone_safety_features():
    return "Safety is a top priority in drone taxi services. These vehicles are equipped with advanced sensors, collision avoidance systems, and redundant mechanisms to ensure passenger safety. Additionally, real-time monitoring and control systems are in place to handle any unexpected situations."

def future_of_drone_taxis():
    return "The future of drone taxis holds exciting possibilities. With ongoing advancements in technology, we can expect improvements in efficiency, longer flight durations, and widespread integration into urban transportation systems. Imagine a world where you can hail a drone taxi with a mobile app!"

def main():
    print("Chatbot Menu:")
    print("1. Drone Working Hours")
    print("2. Drone Price")
    print("3. Drone Taxi Services")
    print("4. Drone Safety Features")
    print("5. Future of Drone Taxis")
    print("0. Start Chatting")
    print("q. Quit")

    while True:
        user_input = input("Select an option (0-5, q): ")

        if user_input.lower() == 'q':
            print("Quitting the program. Goodbye!")
            break
        elif user_input == '0':
            print("You are now chatting with the chatbot. Type 'exit' to end the chat.")
            while True:
                user_input = input("You: ")
                if user_input.lower() == 'exit':
                    print("Exiting the chat. Goodbye!")
                    break

                response = chatbot_response(user_input)
                print("Chatbot:", response)
        elif user_input == '1':
            response = drone_working_hours()
            print("Chatbot:", response)
        elif user_input == '2':
            response = drone_price()
            print("Chatbot:", response)
        elif user_input == '3':
            response = drone_taxi_services()
            print("Chatbot:", response)
        elif user_input == '4':
            response = drone_safety_features()
            print("Chatbot:", response)
        elif user_input == '5':
            response = future_of_drone_taxis()
            print("Chatbot:", response)
        else:
            print("Invalid selection. Please choose a valid option (0-5, q).")

if __name__ == "__main__":
    main()

Chatbot Menu:
1. Drone Working Hours
2. Drone Price
3. Drone Taxi Services
4. Drone Safety Features
5. Future of Drone Taxis
0. Start Chatting
q. Quit


Select an option (0-5, q):  0


You are now chatting with the chatbot. Type 'exit' to end the chat.


You:  안잔


Chatbot: 안녕하세요! 질문이 있으신가요?


...

Chatbot: 가격은 경로, 시간 및 수요에 따라 다를 수 있습니다. 정확한 정보를 원하시면 직접 문의해주세요.


You:  exit


Exiting the chat. Goodbye!


Select an option (0-5, q):  q


Quitting the program. Goodbye!
